In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import time
import string
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [72]:
#Quantidade maxima de páginas a serem extraidas
no_pages = 2

#Termo de busca digitado pelo user
search = "dan+brown"

def get_data(pageNo, searchTitle):  
    #Configuracao do BeatifulSoup
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    # MODELO DE URL DO SITE - https://www3.livrariacultura.com.br/livros/?ft=dan+brown&originalText=dan+brown#1
    r = requests.get('https://www3.livrariacultura.com.br/livros/?ft='+str(searchTitle)+'&originalText='+str(searchTitle)+'#' + str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []

    #Encontra no html todas as lis com essa class e pega seu conteúdo
    for d in soup.findAll('li', attrs={'class':'livros'}):
        #print(d)
        name = d.find('h2', attrs={'class':'prateleiraProduto__informacao__nome'})
        name = name.text
        name = re.sub('\n','', name)
        name = string.capwords(name)

        #n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('li')
        author = author.text
        author = re.split(r'\|', author)
        
        for i in range(len(author)):
            if re.search(r'Autor:', author[i]):
                author = author[i]
                break
        author = re.sub(r'Autor:','', author)
        author = re.split(r'\, ', author)
        if len(author) >= 2:
            author = author[1] + " " + author[0]
        else:
            author = author[0]
        author = string.capwords(author)
        #rating = d.find('span', attrs={'class':'4,1 de 5 estrelas'})
        #r = rating.find_all('span', aria-label=True)
        
        price = d.find('span', attrs={'class':'prateleiraProduto__informacao__preco--valor'})

        imgUrl = d.find('div', attrs={'class': 'prateleiraProduto__foto__content'}).find('template', attrs={'class':'foto__template__image'}).find('img')['src']
        
        siteUrl = d.find('h2', attrs={'class':'prateleiraProduto__informacao__nome'}).find('a')['href']

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(name)
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')
        
        if imgUrl is not None:
            all1.append(imgUrl)
        else:
            all1.append("invalid-image")
        
        if siteUrl is not None:
            all1.append(siteUrl)
        else:
            all1.append("invalid-url-site")

        #if rating is not None:
            #print(rating.text)
        #    all1.append(rating.text)
        #else:
        #    all1.append('-1')

        #if users_rated is not None:
            #print(price.text)
        #    all1.append(users_rated.text)
        #else:
        #    all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [73]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i, search))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor', 'ImgUrl', 'SiteUrl', 'Preco'])
df.to_csv('livrariacultura_products.csv', index=False, encoding='utf-8')

In [74]:
df = pd.read_csv("livrariacultura_products.csv")
df.shape

(48, 5)

In [75]:
df.head(61)

,Titulo,Autor,ImgUrl,SiteUrl,Preco
0,Sinfonia Dos Animais,Dan Brown,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/sinfonia-d...,"R$ 49,90"
1,As Musas,Alex Michaelides,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/as-musas-2...,"R$ 49,90"
2,Mais Forte Que O Sol (irmãs Lyndon – Livro 2),Julia Quinn,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/mais-forte...,"R$ 44,90"
3,Origem,Dan Brown,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/origem-211...,"R$ 36,90"
4,O Símbolo Perdido,Dan Brown,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/o-simbolo-...,"R$ 39,90"
5,O Código Da Vinci,Dan Brown,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/o-codigo-d...,"R$ 39,90"
6,Pelos Caminhos Da História,Vasco Mariz,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/pelos-cami...,"R$ 59,90"
7,Fedro,Platão,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/fedro-2000...,"R$ 69,00"
8,Terceira Humanidade (vol. 1),Bernard Weber,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/terceira-h...,"R$ 79,90"
9,O Portal,Laura Ruby,https://livrariacultura.vteximg.com.br/arquivo...,https://www3.livrariacultura.com.br/o-portal-7...,"R$ 27,45"
